In [0]:
%run "../0.includes/configuration"

In [0]:
%run "../0.includes/utils"

In [0]:
%run "../1.setup/incremental_load_preparation"

### Read and save data into tables

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, FloatType
from pyspark.sql.functions import current_timestamp, lit, date_trunc, trunc,from_utc_timestamp

In [0]:
products_schema = StructType([StructField("sku", IntegerType(), False), 
                              StructField("category", StringType(), True),
                              StructField("name", StringType(), True),
                              StructField("price", DoubleType(), True),
                              StructField("sold", IntegerType(), True),
                              StructField("description", StringType(), True)
                            ])


     

products_df = spark.read \
    .option("header", True) \
    .schema(products_schema) \
    .csv(f"{RAW_FOLDER_PATH}/Products_20250120.csv")

In [0]:
products_df.printSchema()

### Add `ingestion_date column`and `data_source` column

In [0]:
updated_products_df = add_ingestion_date(products_df)
final_products_df = add_data_source(updated_products_df)
display(final_products_df)

In [0]:
final_products_df.write.format("delta").mode("overwrite").saveAsTable("bronze.sales")

In [0]:
%sql
SELECT * 
FROM bronze.sales
LIMIT 5

In [0]:
%sql 
SELECT COUNT(*)
FROM bronze.sales